#### Introduction

Django is a high-level Python Web framework that encourages rapid development and clean, pragmatic design. Built by experienced developers, it takes care of much of the hassle of Web development, so you can focus on writing your app without needing to reinvent the wheel. It’s free and open source.

When you create an app with Django, the default settings are geared towards fast local development.  

 

Defaults are: 

- SQLite 

- Local webserver 

- Local static asset hosting 

- Built-in User model 

- Debug mode turned on 

 

We will use options that are aligned with Django community best practices, for building a proffesional website. 

This includes: <br>
- Docker for local development and deployment
- PostgreSQL
- Custom user model
- Robust user authentication flow with email
- comprehensive testing
- environment variables
- security and performance improvements and more. 

<br>

#### Contents
1.1 Getting Started


#### 1.1 Creating your project

In the command prompt create a folder in your desired destination and navigate to that folder.

Following this you install the required libaries, enter the virtual environment, and start your django project <br>

```
C:\Users\Viraj\Work>mkdir project_folder && cd project_folder

pipenv install django=2.2.3 

pipenv shell 

django-admin startproject <name_of_project> . 
```

Django starts with SQLite for your default website, however this is not scalable and efficient for a productionised website. This notebook will assume you are using a database suitable for production such as postgresql.

Django-admin startproject will kickstart your project with neccesary files needed for your project. You will slowly learn the value of each of these files as we progress through the project.

<img src="Images/django startproject.PNG" width = "1500"/>


#### 1.2 Docker

We will immediately create a docker container and work within here. This also prevents backtracking as your local development may not behave the same within the container. <br>

Properly configuring a local development environment remains a steep challenge despite all the other advances as we could have: 

- A different computer 

- Operating System 

- Versions of Django 

- Virtual Environment options 



In your code editor, create a dockerfile and docker-compose.yml file.

<img src="Images/cp12dockerfile.PNG" width = "1500"/>

#### Your dockerfile should contain: <br>

```
FROM python:3.7-slim

# Sets environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONBUFFERED 1

# Sets working dir, preventing need for a long code path when running commands in docker
WORKDIR /code

# Copy across pipfile and pipfile.lock, and install the dependencies
COPY Pipfile Pipfile.lock /code/
RUN pip install pipenv && pipenv install --system

# Copy across the remainding code
COPY . /code/
```

#### Your docker-compose.yml file should contain:
```
version: '3.7' 


services: 

  web: 

    build: .                                            

    command: python /code/manage.py runserver 0.0.0.0:8000 

    volumes: 

      - .:/code  

    ports: 

      - 8000:8000 

    depends_on: 

     - db 

 
 

  db: 

    image: postgres:11 

    environment: 

      - POSTGRES_PASSWORD=somepassword 
      ```

finally navigate to the folder working directory, and run the following commands:

```
docker build .
docker-compose up

```

To test this has worked correctly, navigate to http://127.0.0.1:8000/ and you should see the following screen:

DJANGO IMAGE PASTED HERE

If you recieve an error, it's likely due to the fact the project files haven't been changed to recognise postgresql as the database of choice.




#### 1.3 Django Project - Changing to postgresql db

There are certain changes that must be undertaken for your django project to recognize and use the postgresql database. <br>

1. The settings file should be adjusted as seen below:

DATABASES = { 

    'default': { 

        'ENGINE': 'django.db.backends.postgresql', 

        'NAME': 'postgres', 

        'USER': 'postgres', 

        'PASSWORD' : 'somepassword', #same password in environment variable 

        'HOST': 'db',  

        'PORT': 5432   

    } 

}

2. pyscopg must be installed 
Psycopg is the most popular PostgreSQL database adapter for the Python programming language. Its main features are the complete implementation of the Python DB API 2.0 specification and the thread safety.

if your docker container is not running, enter the following command in your command prompt to start the docker container and to install the new required libary:
```
docker-compose up
docker-compose exec web pipenv --python /usr/bin/python3 install psycopg2-binary==2.8.3 

```
Please change the following section if python isn't recognised
(--python YOURFILEPATHTOPYTHON). You can also run the following line to check where python is installed on your docker container if you're unsure:
```
docker-compose exec web which python
```

3.  Migrtate and Create superuser
Migrate synchronizes the database state with the current set of models and migrations. Migrations, their relationship with apps and more are covered in depth in the migrations documentation.
https://docs.djangoproject.com/en/3.1/topics/migrations/
